<a href="https://colab.research.google.com/github/vaegev/tensorflow_basics/blob/master/ann_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load Data

In [1]:
import tensorflow as tf
import numpy as np

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

11493376/11490434 [==============================] - 0s 0us/step


# Estimator High-Level API

In [2]:
feature_cols = [tf.feature_column.numeric_column("X", shape=[28*28])]
dnn_clf = tf.estimator.DNNClassifier(hidden_units = [300, 100], 
                           feature_columns = feature_cols, 
                           n_classes = 10)
input_fn = tf.estimator.inputs.numpy_input_fn(x={"X": X_train}, y=y_train, 
                                              batch_size=50, 
                                              num_epochs=40, 
                                              shuffle=True)
dnn_clf.train(input_fn=input_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpra1h8put', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff974198dd8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, u

In [3]:
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"X": X_test}, y=y_test, shuffle=False)
eval_results = dnn_clf.evaluate(input_fn=test_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-01-27-13:28:27
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpra1h8put/model.ckpt-44000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-01-27-13:28:28
INFO:tensorflow:Saving dict for global step 44000: accuracy = 0.9773, average_loss = 0.10717947, global_step = 44000, loss = 13.567021
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 44000: /tmp/tmpra1h8put/model.ckpt-44000


In [4]:
eval_results

{'accuracy': 0.9773,
 'average_loss': 0.10717947,
 'global_step': 44000,
 'loss': 13.567021}

In [5]:
y_pred_iter = dnn_clf.predict(input_fn=test_input_fn)
y_pred = list(y_pred_iter)
y_pred[0]

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpra1h8put/model.ckpt-44000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


{'class_ids': array([7]),
 'classes': array([b'7'], dtype=object),
 'logits': array([ -8.497271  ,  -1.5275851 ,   0.26901457,   7.9783134 ,
         -9.098333  ,  -7.7936296 , -25.056866  ,  23.301195  ,
         -1.5286586 ,   2.3171153 ], dtype=float32),
 'probabilities': array([1.5491796e-14, 1.6481521e-11, 9.9368999e-11, 2.2149143e-07,
        8.4930469e-15, 3.1310407e-14, 9.9623121e-22, 9.9999976e-01,
        1.6463863e-11, 7.7042445e-10], dtype=float32)}

# Using plain TensorFlow

## Construction Phase

In [0]:
n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

In [0]:
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

In [0]:
def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2 / np.sqrt(n_inputs)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
        W = tf.Variable(init, name='kernel')
        b = tf.Variable(tf.zeros([n_neurons]), name='bias')
        Z = tf.matmul(X, W) + b
        if activation is not None:
            return activation(Z)
        else:
            return Z

In [0]:
with tf.name_scope("dnn"):
    hidden1 = neuron_layer(X, n_hidden1, name="hidden1", activation=tf.nn.relu)
    hidden2 = neuron_layer(hidden1, n_hidden2, name="hidden2", 
                           activation=tf.nn.relu)
    logits = neuron_layer(hidden2, n_outputs, name="outputs")

In [0]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,
                                                              logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

In [0]:
learning_rate = 0.01
with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(loss)

In [0]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [0]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [0]:
n_epochs = 40
batch_size = 50

In [0]:
def shuffle_batch(X, y, batch):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X)
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

In [37]:
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "Batch accuracy:", acc_batch, "Val accuracy:", acc_val)

    save_path = saver.save(sess, "./my_model_final.ckpt")

0 Batch accuracy: 1.0 Val accuracy: 0.9684
1 Batch accuracy: 1.0 Val accuracy: 0.9736
2 Batch accuracy: 1.0 Val accuracy: 0.9744
3 Batch accuracy: 1.0 Val accuracy: 0.9782
4 Batch accuracy: 1.0 Val accuracy: 0.9764
5 Batch accuracy: 1.0 Val accuracy: 0.9792
6 Batch accuracy: 1.0 Val accuracy: 0.9778
7 Batch accuracy: 1.0 Val accuracy: 0.981
8 Batch accuracy: 1.0 Val accuracy: 0.9802
9 Batch accuracy: 1.0 Val accuracy: 0.9804
10 Batch accuracy: 1.0 Val accuracy: 0.9798
11 Batch accuracy: 1.0 Val accuracy: 0.977
12 Batch accuracy: 1.0 Val accuracy: 0.9802
13 Batch accuracy: 1.0 Val accuracy: 0.9814
14 Batch accuracy: 1.0 Val accuracy: 0.9792
15 Batch accuracy: 1.0 Val accuracy: 0.9804
16 Batch accuracy: 1.0 Val accuracy: 0.9798
17 Batch accuracy: 1.0 Val accuracy: 0.9794
18 Batch accuracy: 1.0 Val accuracy: 0.98
19 Batch accuracy: 1.0 Val accuracy: 0.9822
20 Batch accuracy: 1.0 Val accuracy: 0.9818
21 Batch accuracy: 1.0 Val accuracy: 0.9828
22 Batch accuracy: 1.0 Val accuracy: 0.9812
23

In [38]:
with tf.Session() as sess:
    saver.restore(sess, "./my_model_final.ckpt") # or better, use save_path
    X_new_scaled = X_test[:20]
    Z = logits.eval(feed_dict={X: X_new_scaled})
    y_pred = np.argmax(Z, axis=1)


INFO:tensorflow:Restoring parameters from ./my_model_final.ckpt


In [39]:
print("Predicted classes:", y_pred)
print("Actual classes:   ", y_test[:20])

Predicted classes: [7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4]
Actual classes:    [7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4]


## tf.layers.dense

In [0]:

n_inputs = 28*28  # MNIST
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

In [0]:
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

In [0]:
with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1",
                              activation=tf.nn.relu)
    hidden2 = tf.layers.dense(hidden1, n_hidden2, name="hidden2",
                              activation=tf.nn.relu)
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")
    y_proba = tf.nn.softmax(logits)

In [0]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

In [0]:
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [0]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [0]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [47]:
n_epochs = 20
n_batches = 50

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_valid = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "Batch accuracy:", acc_batch, "Validation accuracy:", acc_valid)

    save_path = saver.save(sess, "./my_model_final.ckpt")

0 Batch accuracy: 1.0 Validation accuracy: 0.9658
1 Batch accuracy: 1.0 Validation accuracy: 0.9712
2 Batch accuracy: 1.0 Validation accuracy: 0.9768
3 Batch accuracy: 1.0 Validation accuracy: 0.9766
4 Batch accuracy: 1.0 Validation accuracy: 0.9788
5 Batch accuracy: 1.0 Validation accuracy: 0.9788
6 Batch accuracy: 1.0 Validation accuracy: 0.977
7 Batch accuracy: 1.0 Validation accuracy: 0.9784
8 Batch accuracy: 1.0 Validation accuracy: 0.9818
9 Batch accuracy: 1.0 Validation accuracy: 0.9806
10 Batch accuracy: 1.0 Validation accuracy: 0.9828
11 Batch accuracy: 1.0 Validation accuracy: 0.9818
12 Batch accuracy: 1.0 Validation accuracy: 0.9838
13 Batch accuracy: 1.0 Validation accuracy: 0.983
14 Batch accuracy: 1.0 Validation accuracy: 0.983
15 Batch accuracy: 1.0 Validation accuracy: 0.985
16 Batch accuracy: 1.0 Validation accuracy: 0.986
17 Batch accuracy: 1.0 Validation accuracy: 0.987
18 Batch accuracy: 1.0 Validation accuracy: 0.9872
19 Batch accuracy: 1.0 Validation accuracy: 0.9